In [8]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
from nltk.corpus import wordnet 
import nltk
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfFile, PdfPages
from ctypes.wintypes import SIZE
from astropy.timeseries import LombScargle
from matplotlib.backends.backend_pdf import PdfFile, PdfPages
import numpy as np
from astropy.timeseries import LombScargle
import matplotlib.transforms as transforms
from matplotlib.offsetbox import AnchoredText
from scipy.signal import argrelextrema
import time 

In [9]:
def average_word_depth(gcs,start,end): #這邊的start,end單位是no. of timeframe
    endtime = end
    df = pd.read_csv(gcs) 
    if end == "end":
        end = len(df) 
    else:
        end = end+10
    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    
    words = []
    for index, rows in df.loc[(df["Start_time"]>=start) & (df["Start_time"]<=(end))].iterrows():
        words.append(rows["Words"])
    syn_counts = 0
    num_of_hyponyms_below = 0
    worddepths = []
    sum_min_depth = 0
    sum_ratio = 0
    sum_hyponyms = 0 #全部的hyponyms不分階層
    sum_hypernyms = 0 #全部的hypernyms不分階層
    for words_num in range(0, len(words)): #標定字
        if len(wordnet.synsets(words[words_num])) > 0 :
            syns1 = wordnet.synsets(words[words_num])
            syns1_num = len(syns1)#這個字有幾個synset
            syn_counts += syns1_num
            
            for synsnum in range(0,syns1_num):  #把每個synset的depth 都加上去
                #synset_depth_sum+=syns1[synsnum].min_depth()
                ### 用來算到lowest hyponyms的距離
                threshhold = []
                threshhold = syns1[synsnum].hyponyms()  
                hyponyms_layer_count = 0
                quick_ratio = 0
                #print(threshhold)
                #print("word:",syns1[synsnum])
                if threshhold != []:
                    yesorno=True
                else:
                    yesorno = False
                while_count  = 0
                while yesorno:
                    while_count +=1
                    #print("進第",while_count,"層")
                    yesorno = False
                    hyponyms_layer_count += 1
                    new_hyponyms = []
                    #startt = time.time()
                    for syns in threshhold:
                        #print("syns",syns)
                        #print("thresh",threshhold)
                        if syns is not []:
                            yesorno = True 
                            #print("有hypo",syns)
                            new_hyponyms += syns.hyponyms()
                            #print(new_hyponyms)       
                     
                    threshhold = []
                    threshhold = new_hyponyms
                    #print("the",threshhold)
                    new_hyponyms.clear()
                    new_hyponyms = []
            
                
                if hyponyms_layer_count !=0:
                    num_of_hyponyms_below += hyponyms_layer_count
                else:
                    num_of_hyponyms_below +=0
                #print(hyponyms_layer_count)
                if syns1[synsnum].min_depth() != 0 and hyponyms_layer_count != 0:
                    quick_ratio = syns1[synsnum].min_depth()/(syns1[synsnum].min_depth()+hyponyms_layer_count)
                elif syns1[synsnum].min_depth() == 0 and hyponyms_layer_count != 0:
                    quick_ratio = syns1[synsnum].min_depth()/(syns1[synsnum].min_depth()+hyponyms_layer_count)
                elif syns1[synsnum].min_depth() != 0 and hyponyms_layer_count == 0:
                    quick_ratio = syns1[synsnum].min_depth()/(syns1[synsnum].min_depth()+hyponyms_layer_count)
                else: #若是距離root為零
                    quick_ratio = 1
                    #print(syns1[synsnum])
                sum_ratio += quick_ratio
                sum_hypernyms += len(list(set([w for s in syns1[synsnum].closure(lambda s:s.hypernyms()) for w in s.lemma_names()])))
                sum_hyponyms += len(list(set([w for s in syns1[synsnum].closure(lambda s:s.hyponyms()) for w in s.lemma_names()])))
        else:
            continue
    print(num_of_hyponyms_below, syn_counts)
    
    print("start:",start,"end:",endtime, ", hyponyms_layer_count=", round((num_of_hyponyms_below/syn_counts),4),", min_depth_ratio",round(sum_ratio/syn_counts,4),",hyponyms=",sum_hyponyms/syn_counts,",hypernyms=",sum_hypernyms/syn_counts,", root_content_ratio=",sum_hypernyms/(sum_hyponyms+sum_hypernyms),",hypo/hyper=", sum_hyponyms/sum_hypernyms)

In [10]:
syns1 = wordnet.synsets("dog")[0]
vehicle = wordnet.synset('vehicle.n.01')
typesOfVehicles = list(set([w for s in syns1.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
print(len(typesOfVehicles))


278


In [11]:
syns1 = wordnet.synsets("eat")[1]
print(syns1.hyponyms())
vehicle = wordnet.synset('vehicle.n.01')
typesOfVehicles = list(set([w for s in syns1.closure(lambda s:s.hypernyms()) for w in s.lemma_names()]))
print(len(typesOfVehicles))

[Synset('break_bread.v.01'), Synset('breakfast.v.01'), Synset('brunch.v.01'), Synset('dine.v.01'), Synset('eat.v.01'), Synset('eat_in.v.01'), Synset('eat_out.v.01'), Synset('feast.v.01'), Synset('gorge.v.01'), Synset('lunch.v.01'), Synset('mess.v.01'), Synset('nosh.v.01'), Synset('picnic.v.01'), Synset('take_out.v.12'), Synset('victual.v.03')]
5


In [12]:
syns1 = wordnet.synsets("dog")[0]
print(syns1)
print(syns1.closure(lambda s: s.hypernyms()))

Synset('dog.n.01')
<generator object Synset.closure at 0x000001A86C062970>


In [13]:
i=13
print("subject=",i)
average_word_depth(f"data/Text/0{i}(done).csv",0,100)
average_word_depth(f"data/Text/0{i}(done).csv",100,480)

i=18
print("subject=",i)
average_word_depth(f"data/Text/0{i}(done).csv",0,150)
average_word_depth(f"data/Text/0{i}(done).csv",150,480)

i=11
print("subject=",i)
average_word_depth(f"data/Text/0{i}(done).csv",0,300)

i = 10
print("subject=",i)
average_word_depth(f"data/Text/0{i}(done).csv",0,150)
average_word_depth(f"data/Text/0{i}(done).csv",168,300)
average_word_depth(f"data/Text/0{i}(done).csv",300,480)

subject= 13
1140 1436
start: 0 end: 100 , hyponyms_layer_count= 0.7939 , min_depth_ratio 0.7876 ,hyponyms= 16.752785515320333 ,hypernyms= 5.945682451253482 , root_content_ratio= 0.26194201564657155 ,hypo/hyper= 2.8176387912860155


c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('abstraction.n.06') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('entity.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 9
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\read

4228 5333
start: 100 end: 480 , hyponyms_layer_count= 0.7928 , min_depth_ratio 0.7716 ,hyponyms= 49.36958559909994 ,hypernyms= 5.605850365647853 , root_content_ratio= 0.10197009386596813 ,hypo/hyper= 8.806796895905807
subject= 18


c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('lip.n.01') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('tongue.n.01') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('umbilical_cord.n.01') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('diaphragm.n.02') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:5

1772 2387
start: 0 end: 150 , hyponyms_layer_count= 0.7424 , min_depth_ratio 0.809 ,hyponyms= 41.18391286133222 ,hypernyms= 6.300377042312526 , root_content_ratio= 0.13268340023821076 ,hypo/hyper= 6.5367378150142965


...

c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('entity.n.01') at depth 14
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('judo.n.01') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('auscultation.n.01') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('gambling.n.01') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:59

1396 1920
start: 168 end: 300 , hyponyms_layer_count= 0.7271 , min_depth_ratio 0.7898 ,hyponyms= 41.510416666666664 ,hypernyms= 5.3140625 , root_content_ratio= 0.11348898257010333 ,hypo/hyper= 7.811428011369205


c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('hand_shovel.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('plane.n.05') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('entity.n.01') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('limestone.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:59

1694 2164
start: 300 end: 480 , hyponyms_layer_count= 0.7828 , min_depth_ratio 0.7804 ,hyponyms= 43.49445471349353 ,hypernyms= 5.913123844731977 , root_content_ratio= 0.11968050281524159 ,hypo/hyper= 7.355579868708972


c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\nltk\corpus\reader\wordnet.py:599: UserWarning: Discarded redundant search for Synset('entity.n.01') at depth 14
  for synset in acyclic_breadth_first(self, rel, depth):


In [14]:
syns1 = wordnet.synsets("college_student")
print(syns1[0].hyponyms())

[Synset('matriculate.n.01'), Synset('student_teacher.n.01')]
